In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

## 3D meshing example

This notebook shows how to

1. Load and visualize a volume
1. Apply image filters to and segment image
1. Generate a 3D surface mesh from the binary image
1. Decimate/simplify a 3D mesh
1. Visualize and save the mesh to gmsh22 format

### Todo:

- Generate 3D volume mesh
- Surface mesh must be 'watertight'
- Constrained Delaunay triangulation to fix surface (do not go outside surface), i.e.
https://notebook.community/daniel-koehn/Theory-of-seismic-waves-II/02_Mesh_generation/4_Tri_mesh_delaunay_yigma_tepe,
https://wias-berlin.de/software/tetgen/1.5/doc/manual/manual002.html#sec7
- For padding, keep tetrahedron if one of the points is inside the boundaries

In [ ]:
from nanomesh.volume import Volume
import pyvista as pv
from skimage import filters
import numpy as np

In [ ]:
vol = Volume.load('sample_data.npy')

vol.image = vol.image[50:100, 50:100, 50:100]

vol_gauss = vol.apply(filters.gaussian, sigma=5)

thresh = vol_gauss.apply(filters.threshold_li)

seg_vol = vol_gauss.apply(np.digitize, bins=[thresh])

seg_vol.show_slice()

### Generate 3d tetragonal mesh

Meshes can be generated using the `Mesher3D` class.

A convenience method is also available through `volume.generate_mesh`, which offers less flexibility.

In [ ]:
import gc
gc.collect()

In [ ]:
%%time

from nanomesh.mesh3d import Mesher3D

mesher = Mesher3D(seg_vol.image)
mesher.pad(pad_width=2)
mesher.add_points(point_density=1/500, step_size=1, label=1)
mesher.generate_surface_mesh(step_size=2)
mesher.smooth_mesh()
mesher.simplify_mesh(n_faces=5000)
mesher.generate_volume_mesh()
mesher.generate_domain_mask()
mesh = mesher.to_meshio()

pv.plot_itk(mesh)

Show subsection of the tetragonal mesh.

In [ ]:
from nanomesh.mesh3d import show_submesh

show_submesh(mesh, index=30)